# AIB DataHack

Ok let's give this machine learning malarkey a go...

## Data Import and Initial Prep

In [13]:
# library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [14]:
# data import
train = pd.read_csv('data/training_small_processed.csv')
train.shape

(4999, 82)

In [15]:
train.head()

,BuildingID,Year,EnergyRatingCat,EnergyRatingCont,GroundFloorArea,AvgWallU,AvgRoofU,AvgFloorU,AvgWindowU,AvgDoorU,...,ProdFirstEnerDelivered,ProdFirstEnerConvFac,ProdSecondEnerDelivered,ProdSecondEnerConvFac,ProdThirdEnerDelivered,ProdThirdEnerConvFac,PrimaryEnergySecondarySpace,EnergyMainWater,MainSHFuel_cat,MainWHFuel_cat
0,190835,1890,F,418.42,52.04,2.10,2.30,0.61,2.71,3.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,2335.6809,3,4
1,523113,1901,G,520.35,147.44,2.01,0.75,0.94,2.63,3.960000,...,0.0,0.0,0.0,0.0,0.0,0.0,7144.838,4470.0759,3,4
2,271393,1968,D2,281.33,104.84,0.38,0.13,0.68,2.81,2.482515,...,0.0,0.0,0.0,0.0,0.0,0.0,2127.347,5837.4729,3,4
3,585961,2002,C2,188.17,95.31,0.55,0.26,0.34,2.80,3.030000,...,0.0,0.0,0.0,0.0,0.0,0.0,2771.886,3769.6248,5,6
4,624083,2002,B3,134.95,283.87,0.55,0.26,0.41,3.10,3.010000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,5296.5828,5,6


In [16]:
target = train['EnergyRatingCat']
del train['EnergyRatingCat']
del train['EnergyRatingCont']
del train['MainSHFuel']
del train['MainWHFuel']
target.head()

0    F 
1    G 
2    D2
3    C2
4    B3
Name: EnergyRatingCat, dtype: object

## First Model